Integrate a Vector Database

In [6]:
!pip install faiss-cpu


  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl (30.7 MB)


In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Sample agricultural data
documents = [
    # Rice Crop Health Monitoring
    "How to detect rice leaf diseases using deep learning?",
    "Best machine learning techniques for rice yield prediction",
    "Using satellite imagery for rice crop monitoring",
    "Impact of soil nutrients on rice growth",
    "Real-time pest detection in rice fields using AI",

    # General Agricultural Applications
    "Smart irrigation systems using IoT and AI",
    "AI-based weed detection and removal",
    "How climate change affects crop productivity",
    "Precision agriculture techniques for sustainable farming",
    "Using drone technology for crop monitoring",
    "Soil moisture prediction using machine learning",
    "Automated pest detection in agriculture",
    "Enhancing crop rotation planning using AI",
    "Predicting market demand for agricultural products with ML"
]

# Convert text to embeddings
embeddings = model.encode(documents)

# Print embedding shape
print("Embedding shape:", embeddings.shape)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Ensure embeddings are in correct format
embeddings = np.array(embeddings, dtype=np.float32)

# Add to index
index.add(embeddings)
print("FAISS index created and populated.")

# Save index
faiss.write_index(index, "agri_vector.index")
print("Index saved successfully.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding shape: (14, 384)
FAISS index created and populated.
Index saved successfully.


Modify Retrieval Process to Use Vector Database

In [33]:
import os
import faiss
import numpy as np
import google.generativeai as genai
from sentence_transformers import SentenceTransformer

import google.generativeai as genai

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index
index = faiss.read_index("agri_vector.index")

# stored documents (should be dynamically loaded in real use cases)
documents = [
    "How to improve soil fertility?",
    "Best irrigation techniques for rice farming",
    "Effective pest control methods in agriculture",
    "How to increase crop yield efficiently?"
]

# Encode user query
query = "How do I increase crop yield?"
query_embedding = model.encode([query])

# Ensure FAISS-compatible shape
query_embedding = np.array(query_embedding).reshape(1, -1)

# Search in FAISS
D, I = index.search(query_embedding, k=2)  # Retrieve top 2 matches

# Retrieve relevant text
retrieved_text = [documents[i] for i in I[0] if 0 <= i < len(documents)]  # ✅ Avoid out-of-bounds error

# Use retrieved context with Gemini
messages = [
    "You are an expert in agriculture.",
    f"Based on this info: {retrieved_text}, {query}"
]

# 🔥 Call Gemini API
model = genai.GenerativeModel("gemini-1.5-pro-latest")  # Use a valid model
response = model.generate_content(messages)

# Print Gemini's response
print(response.text)


You've provided no specific context about the crop, location, or current practices, which makes recommending specific actions challenging.  Increasing crop yield is a complex issue, but here's a breakdown of general principles and techniques an expert would consider, followed by questions to help you tailor the advice to *your* situation:

**Fundamental Principles for Increasing Crop Yield:**

1. **Plant Nutrition:**
    * **Soil Testing:** Absolutely essential.  Identifies nutrient deficiencies and informs fertilizer choices.
    * **Balanced Fertilization:**  Provide the right nutrients (N, P, K, micronutrients) in the right amounts and at the right time. Consider slow-release fertilizers or split applications to minimize losses. Foliar feeding can address specific deficiencies quickly.
    * **Organic Matter Management:**  Improves soil structure, water retention, and nutrient availability.  Use cover crops, compost, and manure.

2. **Water Management:**
    * **Irrigation:**  Provi